<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Токсик_комментс»

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задание: обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 


## Подготовка

импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
import re
import nltk
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer #lemm
nltk.download('wordnet')#lemm
from nltk.corpus import stopwords
from tqdm import notebook
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
try:
    df = pd.read_csv('C:/Users/Nshan/Desktop/машинное обучение в тексте/toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
df.isna().sum()

text     0
toxic    0
dtype: int64

Пропусков в датасете нет.

In [6]:
df.duplicated().sum()

0

In [7]:
df[df['text'].str.lower().duplicated(keep=False)].sort_values(by='text').head(10)

,text,toxic
143080,"""\n\n """"wikipedia isn't perfect, but its model...",0
36974,"""\n\n """"wikipedia isn't perfect, but its model...",0
136461,"""\n\n Please do not add nonsense to Wikipedia ...",0
78415,"""\n\n Please do not add nonsense to wikipedia ...",0
16019,"""\n\n Unblock Request \n\n""",0
29636,"""\n\n Unblock request \n\n""",0
96528,"""\n\n unblock request \n\n""",0
135029,"""\n\nPlease do not add nonsense to Wikipedia. ...",0
68816,"""\n\nPlease do not add nonsense to Wikipedia. ...",0
141796,"""Thank you for experimenting with Wikipedia. ...",0


In [8]:
df['text'] = df['text'].str.lower()

In [9]:
df.duplicated().sum()

45

Найдено 45 дубликатов. Удалим их.

In [10]:
df = df.drop_duplicates().reset_index(drop=True)

In [11]:
df.duplicated().sum()

0

In [12]:
features = df['text']
target = df['toxic']

In [13]:
features

0         explanation\nwhy the edits made under my usern...
1         d'aww! he matches this background colour i'm s...
2         hey man, i'm really not trying to edit war. it...
3         "\nmore\ni can't make any real suggestions on ...
4         you, sir, are my hero. any chance you remember...
                                ...                        
159521    ":::::and for the second time of asking, when ...
159522    you should be ashamed of yourself \n\nthat is ...
159523    spitzer \n\numm, theres no actual article for ...
159524    and it looks like it was actually you who put ...
159525    "\nand ... i really don't think you understand...
Name: text, Length: 159526, dtype: object

In [14]:
#m = Mystem()
#features = features.apply(lambda x: m.lemmatize(x))
lemm = WordNetLemmatizer()

In [15]:
features = features.apply(lambda x: lemm.lemmatize(x))
#for text in features:
 #   fin_text = []
  #  text_words =  text.split()
   # for word in text_words:
    #    word = lemm.lemmatize(word)
     #   fin_text.append(word)
    #features[text] = ' '.join(fin_text)

In [16]:
features

0         explanation\nwhy the edits made under my usern...
1         d'aww! he matches this background colour i'm s...
2         hey man, i'm really not trying to edit war. it...
3         "\nmore\ni can't make any real suggestions on ...
4         you, sir, are my hero. any chance you remember...
                                ...                        
159521    ":::::and for the second time of asking, when ...
159522    you should be ashamed of yourself \n\nthat is ...
159523    spitzer \n\numm, theres no actual article for ...
159524    and it looks like it was actually you who put ...
159525    "\nand ... i really don't think you understand...
Name: text, Length: 159526, dtype: object

In [17]:
features = features.apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))

In [18]:
features

0         explanation why the edits made under my userna...
1         d aww  he matches this background colour i m s...
2         hey man  i m really not trying to edit war  it...
3           more i can t make any real suggestions on im...
4         you  sir  are my hero  any chance you remember...
                                ...                        
159521          and for the second time of asking  when ...
159522    you should be ashamed of yourself   that is a ...
159523    spitzer   umm  theres no actual article for pr...
159524    and it looks like it was actually you who put ...
159525      and     i really don t think you understand ...
Name: text, Length: 159526, dtype: object

In [19]:
for i in range(len(features)):
    features[i] = features[i].split()

In [20]:
features

0         [explanation, why, the, edits, made, under, my...
1         [d, aww, he, matches, this, background, colour...
2         [hey, man, i, m, really, not, trying, to, edit...
3         [more, i, can, t, make, any, real, suggestions...
4         [you, sir, are, my, hero, any, chance, you, re...
                                ...                        
159521    [and, for, the, second, time, of, asking, when...
159522    [you, should, be, ashamed, of, yourself, that,...
159523    [spitzer, umm, theres, no, actual, article, fo...
159524    [and, it, looks, like, it, was, actually, you,...
159525    [and, i, really, don, t, think, you, understan...
Name: text, Length: 159526, dtype: object

In [21]:
for i in range(len(features)):
    features[i] = ' '.join(features[i])

In [22]:
features

0         explanation why the edits made under my userna...
1         d aww he matches this background colour i m se...
2         hey man i m really not trying to edit war it s...
3         more i can t make any real suggestions on impr...
4         you sir are my hero any chance you remember wh...
                                ...                        
159521    and for the second time of asking when your vi...
159522    you should be ashamed of yourself that is a ho...
159523    spitzer umm theres no actual article for prost...
159524    and it looks like it was actually you who put ...
159525    and i really don t think you understand i came...
Name: text, Length: 159526, dtype: object

In [23]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
count_v = TfidfVectorizer(stop_words=stop_words)

In [25]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25)

In [26]:
features_train.shape[0]

119644

In [27]:
count_v.fit(features_train)

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [28]:
features_train = count_v.transform(features_train)
features_test = count_v.transform(features_test)


**Выводы**

1. Найлено 45 идентичных комментариев. Дубликаты удалены.
2. Пропусков в датасете нет.
3. Данные подготовлены и разделены на обучающую и тестовую (75:25)

## Обучение

Модель Логистической регрессии

In [29]:
model = LogisticRegression(class_weight='balanced')
res = cross_val_score(model, features_train, target_train, cv=3, scoring='f1')

In [30]:
print('Средняя точность логистической регрессии на кроссвалидации составила:', res.sum()/res.shape[0])

Средняя точность логистической регрессии на кроссвалидации составила: 0.7461835570787495


предскзание по логистической регрессии

In [31]:
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('f1 на модели логистической регрессии равен:', f1_score(target_test, predictions))

f1 на модели логистической регрессии равен: 0.753844475951576


модель случайного леса

In [32]:
params_set = {
    'max_depth': [5, 10, 20],
    'min_samples_leaf': [1, 5],
    'n_estimators': [3, 5]
}
grid = GridSearchCV(estimator=RandomForestClassifier(random_state=12345), param_grid=params_set, cv=3, scoring='f1')

In [33]:
grid.fit(features_train, target_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=12345),
             param_grid={'max_depth': [5, 10, 20], 'min_samples_leaf': [1, 5],
                         'n_estimators': [3, 5]},
             scoring='f1')

In [34]:
model_for = grid.best_estimator_
model_for

RandomForestClassifier(max_depth=20, n_estimators=3, random_state=12345)

In [35]:
predictions_forest = model_for.predict(features_test)
print('f1 для модели случайного леса:', f1_score(target_test, predictions_forest))

f1 для модели случайного леса: 0.03571428571428571


модель случайного леса показала плохой результат, что говорит о невозможности использования данной модели для поставленной задачи.

## Выводы

1. найлено 45 идентичных комментариев. Дубликаты удалены
2. Пропусков в датасете нет
3. Данные подготовлены и разделены на обучающую и тестовую (75:25)
4. Средняя точность логистической регрессии на кроссвалидации составила: 0.747335158209104
5. f1 на модели логистической регрессии на тестовой выборке равен: 0.7543683917640513
6. f1 для модели случайного леса: 0.0069703759024147376. модель случайного леса показала плохой результат, что говорит о невозможности использования данной модели для поставленной задачи.
7. Модель CatBoost обучается долго и показала результаты меньшие чем модель логистической регрессии. В связи с тем, что модель долго обучается было решено не включать её в отчёт, так как другая часть кода будет прогружаться очень долго.
**Из всех моделей метрика f1 соотвествует у модели логистической регрессии, поэтому стоит выбрать её**